In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate, cross_val_score
from category_encoders.target_encoder import TargetEncoder
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

random_state = 42

import warnings
warnings.filterwarnings("ignore")

In [2]:
raw_data = pd.read_csv('data_after_feature_selection.csv')

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
raw_data.describe()

,TransactionID,isFraud,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,dist1,C1,C13,D1,D3,D4,D5,D10,D15,V3,V4,V8,V12,V15,V17,V19,V23,V24,V25,V26,V32,V35,V37,V38,V39,V42,V44,V46,V47,V52,V53,V55,V56,V60,V61,V62,V66,V67,V74,V75,V78,V81,V82,V83,V86,V87,V98,V99,V100,V104,V115,V121,V122,V130,V169,V170,V171,V172,V173,V174,V175,V176,V180,V184,V185,V186,V187,V188,V189,V194,V195,V199,V200,V204,V207,V209,V220,V223,V224,V228,V229,V230,V234,V238,V242,V246,V247,V248,V250,V252,V255,V258,V260,V261,V262,V270,V281,V283,V284,V285,V286,V287,V288,V289,V290,V303,V310,V312,V313,V314,id_02,id_05,id_06,id_11,id_13,id_17,id_19,id_20
count,5.905400e+05,590540.000000,5.905400e+05,590540.000000,590540.000000,581607.000000,588975.000000,586281.000000,524834.000000,524834.000000,238269.000000,590540.000000,590540.000000,589271.000000,327662.000000,421618.000000,280699.000000,514518.000000,501427.000000,311253.000000,311253.000000,311253.000000,514467.000000,514467.000000,514467.000000,514467.000000,514467.000000,514467.000000,514467.000000,514467.000000,514467.000000,421571.000000,421571.000000,421571.00000,421571.000000,421571.000000,421571.000000,421571.000000,421571.000000,421571.000000,513444.000000,513444.000000,513444.000000,513444.000000,513444.000000,513444.000000,513444.000000,513444.000000,513444.000000,501376.000000,501376.000000,501376.000000,501376.000000,501376.000000,501376.000000,501376.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,590226.000000,139819.000000,139819.000000,139819.000000,139631.000000,139631.000000,139819.000000,139819.000000,139631.000000,139819.000000,139819.000000,139819.000000,139631.000000,139631.000000,139819.000000,139819.000000,139819.000000,139819.000000,139631.000000,139819.000000,139631.000000,139631.000000,139819.000000,141416.000000,130430.000000,130430.000000,130430.000000,130430.000000,130430.000000,141416.000000,141416.000000,130430.000000,130430.000000,130430.000000,130430.000000,141416.000000,130430.000000,141416.000000,130430.000000,130430.000000,130430.000000,130430.000000,141416.000000,589271.000000,589271.000000,590528.000000,590528.000000,590528.000000,590528.000000,589271.000000,589271.000000,590528.000000,590528.000000,590528.000000,590528.000000,589271.000000,589271.000000,140872.000000,136865.000000,136865.000000,140978.000000,127320.000000,139369.000000,139318.000000,139261.000000
mean,3.282270e+06,0.034990,7.372311e+06,135.027176,9898.734658,362.555488,153.194925,199.278897,290.733794,86.800630,118.502180,14.092458,32.539918,94.347568,28.343348,140.002441,42.335965,123.982137,163.744579,1.078075,0.846456,1.027704,0.559711,0.122342,0.134040,0.816371,1.034791,1.058097,0.977660,0.988040,0.142417,0.542594,1.108065,1.16240,0.166076,0.156118,1.083891,1.022286,1.038515,0.182695,0.577586,1.067670,1.120979,0.142537,0.829785,0.867563,0.981038,0.998121,0.152147,0.544278,1.144462,0.152696,0.844610,0.881965,1.064885,1.099456,0.061985,0.894986,0.273504,0.085433,1.032450,1.004276,1.001759,92.165849,0.168053,1.436779,1.690908,0.132012,0.055231,0.127794,0.212639,1.376979,0.924123,0.132364,0.173903,1.148799,1.843752,1.014755,1.038314,0.945773,0.954227,1.270749,1.119977,686.956931,72.284098,35.001872,0.169472,0.093153,0.394748,1.351928,1.638710,1.465476,2.092847,0.127956,1.113463,1.183723,1.025355,1.069493,0.788588,1.032048,0.806656,1.343510,0.964425,1.107161,1.013279,7.712764,0.087783,0.991114,0.088543,1.167660,0.031492,0.358579,0.184350,0.235975,1.103011,0.283140,118.195658,39.173910,21.351473,43.319174,174716.584708,1.615585,-6.698710,99.745325,48.053071,189.451377,353.128174,403.882666
std,1.704744e+05,0.183755,4.617224e+06,239.162522,4901.170153,157.793246,11.336444,41.244453,101.741072,2.690623,371.872026,133.569018,129.364844,157.660387,62.384721,191.096774,89.000144,182.615225,202.726660,0.320890,0.440053,0.186069,0.510533,0.332422,0.364456,0.425512,0.247681,0.305485,0.185245,0.209302,0.368278,0.516010,0.690571,0.85892,0.451956,0.382

In [5]:
raw_data.describe(include='object')

,ProductCD,card4,P_emaildomain,R_emaildomain,M2,M3,M4,M5,M6,M7,M8,M9,id_12,id_15,id_16,id_28,id_29,id_31,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
count,590540,588963,496084,137291,319440,319440,309096,240058,421180,244275,244288,244288,144233,140985,129340,140978,140978,140282,140985,140985,140985,140985,140810,118666
unique,5,4,59,60,2,2,3,2,2,2,2,2,2,3,2,2,2,130,2,2,2,2,2,1786
top,W,visa,gmail.com,gmail.com,T,T,M0,F,F,F,F,T,NotFound,Found,Found,Found,Found,chrome 63.0,T,F,T,F,desktop,Windows
freq,439670,384767,228355,57147,285468,251731,196405,132491,227856,211374,155251,205656,123025,67728,66324,76232,74926,22000,77814,134066,110452,73922,85165,47722


### Разделим данные на train и test

In [6]:
X_train, X_test = train_test_split(raw_data, test_size=0.25, random_state=random_state)
X_train.reset_index(drop=True, inplace=True)
y_train = X_train['isFraud']
X_train.drop(columns=['isFraud', 'TransactionID'], inplace=True)

X_test.reset_index(drop=True, inplace=True)
y_test = X_test['isFraud']
X_test.drop(columns=['isFraud', 'TransactionID'], inplace=True)

### Заполнение пропусков

In [7]:
class CustomImputer():
    def __init__(self, strategy='most_frequent'):
        self.__imputer = SimpleImputer(strategy=strategy)
        
    def fit(self, X, y=None):
        self.__imputer.fit(X)
        return self
    def transform(self, X, y=None):
        X = X.copy()
        X[X.columns] = self.__imputer.transform(X[X.columns])
        
        return X

### Кодирование категориальных признаков

In [8]:
class ObjectEncoder:
    def __init__(self):
        self.__encoded_columns = None
        self.__targ_enc_cols = ['card1', 'card2', 'card3', 'card5', 'addr1', 'addr2',
                             'id_13', 'id_17', 'id_19', 'id_20', 'id_31', 'DeviceInfo']
        self.__targ_enc = TargetEncoder(cols=self.__targ_enc_cols)
        
    def encode_obj_cols(self, df):
        
        # map binary categorical columns values to 0 or 1
        d_for_map = {'F': 0, 'T': 1, 'NotFound': 0, 'Found': 1, 'New': 0, 'desktop': 0, 'mobile': 1}
        cat_map_cols = ['M1', 'M2', 'M3', 'M5', 'M6', 'M7', 'M8', 'M9',
                        'id_12', 'id_16', 'id_28', 'id_29', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType']
        for col in cat_map_cols:
            if col in df:
                df[col] = df[col].map(d_for_map)
    
        # code One Hot Encoding only cat_dum_cols
        cat_dum_cols = ['card4', 'ProductCD', 'P_emaildomain', 'R_emaildomain', 'M4', 'id_15']
        
        if self.__encoded_columns is not None:
            df = pd.get_dummies(df, drop_first=False, columns=cat_dum_cols)

            removed_cols = []
            for col in df.columns:
                if col not in self.__encoded_columns:
                    removed_cols.append(col)
            df.drop(columns=removed_cols, inplace=True)

            for i in range(len(self.__encoded_columns)):
                col = self.__encoded_columns[i]
                if col not in df:
                    df.insert(i, col, 0)

        else:
            df = pd.get_dummies(df, drop_first=True, columns=cat_dum_cols)
            self.__encoded_columns = df.columns
            
        return df

    def fit(self, X, y):
        self.__targ_enc.fit(X, y)
    
    def transform(self, X):
        X = self.__targ_enc.transform(X)
        return self.encode_obj_cols(X)
    
    def fit_transform(self, X, y):
        X = self.__targ_enc.fit_transform(X, y)
        return self.encode_obj_cols(X)

### Масштабирование числовых признаков

In [9]:
class CustomScaler():
    def __init__(self, cols, scaler=None):
        self.__cols = cols
        self.__scaler = scaler or preprocessing.MinMaxScaler()
        
    def fit(self, X, y=None):
        num_cols = X.copy()[self.__cols]
        self.__scaler.fit(num_cols)
        return self
    def transform(self, X, y=None):
        X_res = X.copy()
        X_res[self.__cols] = self.__scaler.transform(X_res[self.__cols])
        return X_res

In [10]:
# категориальные признаки
cat_cols = ['card1', 'card2', 'card3', 'card5', 'addr1', 'addr2', 'id_13', 'id_17', 'id_19', 'id_20', 'id_31',
            'DeviceInfo', 'M1', 'M2', 'M3', 'M5', 'M6', 'M7', 'M8', 'M9', 'id_12', 'id_16', 'id_28', 'id_29',
            'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType', 'card4', 'ProductCD', 'P_emaildomain', 'R_emaildomain',
            'M4', 'id_15']

In [11]:
# числовые признаки
real_cols = list(raw_data.columns)
real_cols.remove('TransactionID')
real_cols.remove('isFraud')
for col in cat_cols:
    if col in real_cols:
        real_cols.remove(col)

### Обучение модели

In [12]:
# количество уникальных элементов в массиве
def get_unique(y):
    unique, counts = np.unique(y, return_counts=True)
    return np.asarray((unique, counts)).T

In [13]:
# метрики качества
def get_scores(y_test, pred, pred_proba):
    scores = pd.DataFrame({'score name':['accuracy', 'precision', 'recall', 'f1', 'roc-auc'],
                          'score value':[accuracy_score(y_test, pred), precision_score(y_test, pred), 
                                         recall_score(y_test, pred), f1_score(y_test, pred),
                                        roc_auc_score(y_test, pred_proba)]})
    return scores


#### Бейзлайн

In [14]:
baseline_pred = np.zeros(y_test.shape[0])

In [15]:
get_scores(y_test, baseline_pred, baseline_pred)

,score name,score value
0,accuracy,0.964067
1,precision,0.000000
2,recall,0.000000
3,f1,0.000000
4,roc-auc,0.500000


#### Лучший пайплайн с логистической регрессией

In [16]:
p_best = Pipeline([
    ('imputer_', CustomImputer()),
    ('encoder_', ObjectEncoder()),
    ('scaler_', CustomScaler(real_cols)),
    ('model_', LogisticRegression(C=12, class_weight='balanced', max_iter=1000))
])

In [17]:
%%time
p_best.fit(X_train, y_train)
pred = p_best.predict(X_test)

CPU times: total: 14min 9s
Wall time: 4min 13s


In [18]:
print('Train data')
get_scores(y_train, p_best.predict(X_train), p_best.predict_proba(X_train)[:, 1])

Train data


,score name,score value
0,accuracy,0.874544
1,precision,0.180790
2,recall,0.741373
3,f1,0.290693
4,roc-auc,0.894357


In [19]:
print('Test data')
get_scores(y_test, pred, p_best.predict_proba(X_test)[:, 1])

Test data


,score name,score value
0,accuracy,0.871704
1,precision,0.179906
2,recall,0.722337
3,f1,0.288066
4,roc-auc,0.883464
